# Approach   
So given that the request more or less said "DONT USE OVERCOMPLICATED" things, I think I will try to some some lower approaches  


the first plan that did arise is how can I go from data to labels without a LLM or something fancy that I cant say it will perform quite good. And that did take me to a previous task where I needed to create a regression for a word complexity (spoiler it appears that linear regression with word frequency made the best approach there) 

My plan is the following:
- I expand as much as I can the labels -> with synonymy and definitions
- I reduce the size of the data as much as I can 
- Clustering these 2 I think I can create a bridge

In [19]:
import os
import pandas as pd
import nltk
from nltk.corpus import wordnet as wn
import numpy as np

import re
import ast

# these i'll use in order to clen some words out of the description
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer


# for the cluestering part
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

# nltk.download('wordnet')

In [5]:
data = pd.read_csv('input/ml_insurance_challenge.csv')
labels = pd.read_csv('input/insurance_taxonomy - insurance_taxonomy.csv')


print(data.head())
print(data.columns)

                                         description  \
0  Welchcivils is a civil engineering and constru...   
1  Kyoto Vegetable Specialists Uekamo, also known...   
2  Loidholdhof Integrative Hofgemeinschaft is a c...   
3  PATAGONIA Chapa Y Pintura is an auto body shop...   
4  Stanica WODNA PTTK Swornegacie is a cultural e...   

                                       business_tags         sector  \
0  ['Construction Services', 'Multi-utilities', '...       Services   
1  ['Wholesale', 'Dual-task Movement Products', '...  Manufacturing   
2  ['Living Forms', 'Farm Cafe', 'Fresh Coffee', ...  Manufacturing   
3  ['Automotive Body Repair Services', 'Interior ...       Services   
4  ['Cultural Activities', 'Accommodation Service...       Services   

                               category  \
0            Civil Engineering Services   
1  Fruit & Vegetable - Markets & Stores   
2        Farms & Agriculture Production   
3                       Auto Body Shops   
4                  Bo

In [6]:
print(data['sector'].unique())

['Services' 'Manufacturing' 'Wholesale' 'Retail' 'Government' 'Non Profit'
 'Education' nan]


In [7]:
def extract_wordnet(word: str):
    sn =  wn.synsets(word)
    if sn is not None and len(sn) > 0:
        return sn[0].definition()
    else: return ' '

def extract_defition(data: pd.DataFrame):
    data['definitions'] = data['label'].apply(lambda s: ' '.join([extract_wordnet(w) for w in s.split(' ')]))

extract_defition(labels)
print(labels.head())

                                label  \
0     Agricultural Equipment Services   
1  Soil Nutrient Application Services   
2      Pesticide Application Services   
3          Ornamental Plant Nurseries   
4                Landscaping Services   

                                         definitions  
0  relating to or used in or promoting agricultur...  
1  the state of being covered with unclean things...  
2  a chemical used to kill pests (as rodents or i...  
3  any plant grown for its beauty or ornamental v...  
4  working as a landscape gardener performance of...  


In [8]:
def get_syn_hyp(label_text: str):
    words = label_text.split()
    synonyms = set()
    hypernyms = set()

    for w in words:
        syns = wn.synsets(w)
        for syn in syns:
            for lemma in syn.lemmas():
                synonyms.add(lemma.name())

            for hyp in syn.hypernyms():
                for lemma in hyp.lemmas():
                    hypernyms.add(lemma.name())

    return " ".join(synonyms), " ".join(hypernyms)

labels[["synonyms", "hypernyms"]] = labels["label"].apply(
    lambda x: pd.Series(get_syn_hyp(x))
)

In [9]:
print(labels.head())

                                label  \
0     Agricultural Equipment Services   
1  Soil Nutrient Application Services   
2      Pesticide Application Services   
3          Ornamental Plant Nurseries   
4                Landscaping Services   

                                         definitions  \
0  relating to or used in or promoting agricultur...   
1  the state of being covered with unclean things...   
2  a chemical used to kill pests (as rodents or i...   
3  any plant grown for its beauty or ornamental v...   
4  working as a landscape gardener performance of...   

                                            synonyms  \
0  religious_service inspection_and_repair help s...   
1  religious_service dirt inspection_and_repair h...   
2  religious_service inspection_and_repair help p...   
3  baby's_room ornamental nursery implant found e...   
4  religious_service inspection_and_repair help l...   

                                           hypernyms  
0  employment conjugatio

In [12]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def clean_and_extract_essential_words(text: str) -> str:
    text = text.lower()
    
    text = re.sub(r'[^a-z0-9\s]', '', text)
    
    tokens = text.split()
    
    tokens = [
        lemmatizer.lemmatize(tok)
        for tok in tokens
        if tok not in stop_words
    ]
    
    return " ".join(tokens)

def combine_columns(row):
    try:
        tags = ast.literal_eval(row["business_tags"])
    except (SyntaxError, ValueError):
        tags = []

    combined_text = (
        str(row["description"]) + " " +
        " ".join(tags) + " " +
        str(row["sector"]) + " " +
        str(row["category"]) + " " +
        str(row["niche"])
    )
    
    return combined_text

data["domain"] = data.apply(lambda row: clean_and_extract_essential_words(
                                combine_columns(row)
                            ), axis=1)

labels["domain"] = labels["label"].apply(clean_and_extract_essential_words)

print("data head:")
print(data.head())

print("\nlabels head:")
print(labels.head())

print(data.describe)

data head:
                                         description  \
0  Welchcivils is a civil engineering and constru...   
1  Kyoto Vegetable Specialists Uekamo, also known...   
2  Loidholdhof Integrative Hofgemeinschaft is a c...   
3  PATAGONIA Chapa Y Pintura is an auto body shop...   
4  Stanica WODNA PTTK Swornegacie is a cultural e...   

                                       business_tags         sector  \
0  ['Construction Services', 'Multi-utilities', '...       Services   
1  ['Wholesale', 'Dual-task Movement Products', '...  Manufacturing   
2  ['Living Forms', 'Farm Cafe', 'Fresh Coffee', ...  Manufacturing   
3  ['Automotive Body Repair Services', 'Interior ...       Services   
4  ['Cultural Activities', 'Accommodation Service...       Services   

                               category  \
0            Civil Engineering Services   
1  Fruit & Vegetable - Markets & Stores   
2        Farms & Agriculture Production   
3                       Auto Body Shops   
4         

now given that I cant test any accuracy the only thing that I have left I think is to:
- create some manual examples, which I don't really have time for
- create a semi-automate approach where I use the chatgpt approach or a different LLM
- try to use basic clustering metrics like the distance inside the cluster and between the clusters

# Selected approach
Would be the third, for starters and maybe see how can a LLM help me

In [23]:
data_combined = pd.DataFrame({
    "domain": data["domain"],
    "type": "data",
    "original_idx": data.index,
    "original_label_text": None
})
labels_combined = pd.DataFrame({
    "domain": labels["domain"],
    "type": "label",
    "original_idx": labels.index,
    "original_label_text": labels["label"]
})


combined = pd.concat([data_combined, labels_combined], ignore_index=True)
combined = combined.dropna(subset=["domain"])

In [ ]:
vectorizer = TfidfVectorizer(min_df=2, max_df=0.8)
X = vectorizer.fit_transform(combined["domain"])



# now ideally i would have K clusters like the number of labels
num_labels = len(labels_combined)
kmeans = KMeans(n_clusters=num_labels, random_state=42)
clusters = kmeans.fit_predict(X)
combined["cluster"] = clusters


In [ ]:
centroids = kmeans.cluster_centers_

labels_only = combined[combined["type"] == "label"]
cluster_to_label_idx = {} 

for c in range(num_labels):
    cluster_labels = labels_only[labels_only["cluster"] == c]
    
    if len(cluster_labels) == 0:
        cluster_to_label_idx[c] = None
    elif len(cluster_labels) == 1:
        only_index = cluster_labels.index[0]
        cluster_to_label_idx[c] = only_index
    else:
        # one of the problems is when i have too many label in a centroid, then i choose the one
        # closest to the centroid
        label_indices = cluster_labels.index.tolist()
        dist_list = []
        for li in label_indices:
            dist = np.linalg.norm(X[li].toarray() - centroids[c])
            dist_list.append(dist)
        best_label_index = label_indices[np.argmin(dist_list)]
        cluster_to_label_idx[c] = best_label_index

data_only = combined[combined["type"] == "data"]
final_rows = []

for i, row in data_only.iterrows():
    c = row["cluster"]
    label_idx = cluster_to_label_idx[c]
    
    if label_idx is not None:
        assigned_label_text = combined.loc[label_idx, "original_label_text"]
    else:
        assigned_label_text = None
    
    final_rows.append({
        "original_data_index": row["original_idx"],
        "data_domain": row["domain"],
        "cluster": int(c),
        "insurance_label": assigned_label_text
    })

final_df = pd.DataFrame(final_rows)
print("\nPreview of final_df:\n")
print(final_df.head(10))

final_merged = data.merge(
    final_df[["original_data_index", "insurance_label"]], 
    left_index=True, 
    right_on="original_data_index",
    how="left"
)

final_merged.drop(columns=["original_data_index"], inplace=True)

print(final_merged.head())
print(final_merged.columns)


Preview of final_df:

   original_data_index                                        data_domain  \
0                    0  welchcivils civil engineering construction com...   
1                    1  kyoto vegetable specialist uekamo also known i...   
2                    2  loidholdhof integrative hofgemeinschaft compan...   
3                    3  patagonia chapa pintura auto body shop located...   
4                    4  stanica wodna pttk swornegacie cultural establ...   
5                    5  biq benefcios brazilian company specializes ge...   
6                    6  envcompli company provides personalized indust...   
7                    7  corrugated beam form ltd company based walsall...   
8                    8  company specializes production distribution pa...   
9                    9  indira caterz event managerzz company categori...   

   cluster                   insurance_label  
0       28  Commercial Construction Services  
1      112                         

In [36]:

print(final_merged['insurance_label'].isna().sum())

4253


# Now because as the above test suggested not all the rows have a label, which is completely useless in the AI domain, because even though you have good predictions, you can't have 50% missing predictions.


That's why the nex approaches starts by clustering the labels and then the data. If a centroid has multiple labels then we take the closest, after the first batch of asssingments I start to cluster the labels that aren't assigned

In [ ]:
vectorizer = TfidfVectorizer(min_df=1, max_df=0.8)
labels_vector = vectorizer.fit_transform(labels["domain"])

num_labels = len(labels)
kmeans = KMeans(n_clusters=num_labels, random_state=42)
label_clusters = kmeans.fit_predict(labels_vector)
centroids = kmeans.cluster_centers_

cluster_to_label_idx = {}
for c in range(num_labels):
    label_indices_in_cluster = [idx for idx, cluster_id in enumerate(label_clusters) 
                                if cluster_id == c]
    
    if not label_indices_in_cluster:
        cluster_to_label_idx[c] = None
    elif len(label_indices_in_cluster) == 1:
        cluster_to_label_idx[c] = label_indices_in_cluster[0]
    else:
        dist_list = []
        for li in label_indices_in_cluster:
            dist = np.linalg.norm(labels_vector[li].toarray() - centroids[c])
            dist_list.append(dist)
        best_label_index = label_indices_in_cluster[np.argmin(dist_list)]
        cluster_to_label_idx[c] = best_label_index

data_vector = vectorizer.transform(data["domain"])

assigned_labels = []
for i in range(data_vector.shape[0]):
    row_vec = data_vector[i].toarray()
    distances = [np.linalg.norm(row_vec - centroids[c]) for c in range(num_labels)]
    cluster_id = np.argmin(distances)
    best_label_idx = cluster_to_label_idx[cluster_id]
    
    if best_label_idx is not None:
        assigned_labels.append(labels.loc[best_label_idx, "label"])
    else:
        assigned_labels.append(None)

data["insurance_label"] = assigned_labels

print(data)

                                            description  \
0     Welchcivils is a civil engineering and constru...   
1     Kyoto Vegetable Specialists Uekamo, also known...   
2     Loidholdhof Integrative Hofgemeinschaft is a c...   
3     PATAGONIA Chapa Y Pintura is an auto body shop...   
4     Stanica WODNA PTTK Swornegacie is a cultural e...   
...                                                 ...   
9489  Anhui Zhongxin Electric Co., Ltd. is a high-te...   
9490  TP Material Co.,Ltd. is a company based in the...   
9491  Aladiner Cherag is a company that offers a var...   
9492  Candor Eeg is a medical care company located i...   
9493  REKA Vertrieb Deutschland is a company based i...   

                                          business_tags         sector  \
0     ['Construction Services', 'Multi-utilities', '...       Services   
1     ['Wholesale', 'Dual-task Movement Products', '...  Manufacturing   
2     ['Living Forms', 'Farm Cafe', 'Fresh Coffee', ...  Manufacturin

In [ ]:
print(data['insurance_label'].isna().sum())

0


In [40]:
data.to_csv('output/ml_insurance_challenge_solved.csv')

In [20]:
labels['definitions'].to_csv('testing.csv')